# Advent of Code 2024: Day 23
https://adventofcode.com/2024/day/23


## Part 1
Get the longest path from the start to the end

### Get the data into a list of lists of characters

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
data_map = [list(line) for line in data_list]

### Create function to retrieve the valid neighbors for a given position

In [2]:
def getNeighbors(pos, data):
  nrows = len(data)
  ncols = len(data[0])
  row = pos[0]
  col = pos[1]
  neighbors = []

  if (row > 0) and (data[row-1][col] not in ['#','v']) and (data[row][col] not in ['<','>','v']):
    neighbors.append((row-1, col))

  if (row < nrows-1) and (data[row+1][col] not in ['#','^']) and (data[row][col] not in ['<','>','^']):
    neighbors.append((row+1, col))

  if (col > 0) and (data[row][col-1] not in ['#','>']) and (data[row][col] not in ['v','>','^']):
    neighbors.append((row, col-1))

  if (col < ncols-1) and (data[row][col+1] not in ['#','<']) and (data[row][col] not in ['<','v','^']):
    neighbors.append((row, col+1))

  return neighbors

### Create function to perform a bfs to find the longest path

In [3]:
def longestPath(start,end,data):
  explore_q = [start]
  prev = {start: [start]}
  dist = {start: [0]}
  visited = set([start])
  while explore_q:
    cur = explore_q.pop(0)
    neighbors = getNeighbors(cur, data)
    for neighbor in neighbors:
      if neighbor not in visited:
        visited.add(neighbor)
        explore_q.append(neighbor)
        prev[neighbor] = [cur]
        dist[neighbor] = [dist[cur][-1]+1]
      else:
        if neighbor not in prev[neighbor] and neighbor not in prev[cur]:
          prev[neighbor].append(cur)
          dist[neighbor].append(dist[cur][-1]+1)

  cur = end
  longest_path = [cur]
  while cur not in prev[cur]:
    if len(prev[cur]) == 1:
      longest_path.insert(0, prev[cur][0])
      cur = prev[cur][0]
    else:
      idx = dist[cur].index(max(dist[cur]))
      longest_path.insert(0, prev[cur][idx])
      cur = prev[cur][idx]
  return len(longest_path)-1

### Find the length of the longest path

In [4]:
start = (0,1)
end = (len(data_map)-1, len(data_map[0])-2)
longestPath(start, end, data_map)

2086

## Part 2
Get the longest path from the start to the end, new map rules

### Create new function to retrieve the valid neighbors for a given position

In [5]:
def getNeighbors2(pos, data):
  nrows = len(data)
  ncols = len(data[0])
  row = pos[0]
  col = pos[1]
  neighbors = []

  if (row > 0) and (data[row-1][col] != '#'):
    neighbors.append((row-1, col))

  if (row < nrows-1) and (data[row+1][col] != '#'):
    neighbors.append((row+1, col))

  if (col > 0) and (data[row][col-1] !='#'):
    neighbors.append((row, col-1))

  if (col < ncols-1) and (data[row][col+1] != '#'):
    neighbors.append((row, col+1))

  return neighbors

### Create modified version of a path-finding algorithm
Try to reach the end, but only if it's a single path. If a split in the road is found before reaching the end, stop early and return None.

In [6]:
def path(start, end, data):
  explore_q = [start]
  prev = {start: start}
  dist = {start: 0}
  visited = set([start])
  while explore_q:
    cur = explore_q.pop(0)
    neighbors = getNeighbors2(cur, data)
    if len(neighbors) <= 2 or cur == start:
      for neighbor in neighbors:
        if neighbor not in visited:
          visited.add(neighbor)
          explore_q.append(neighbor)
          prev[neighbor] = cur
          dist[neighbor] = dist[cur]+1
          if neighbor == end:
            return dist[neighbor]
    else:
      continue
  return None

### Create function to retrieve all positions where the path splits

In [7]:
def getSplits(data):
  splits = set()
  for i in range(len(data)):
    for j in range(len(data[0])):
      if data[i][j] == '.':
        if len(getNeighbors2((i,j),data)) > 2:
          splits.add((i,j))
  return splits

### Create function to retrieve a graph of the splits
For each split (as well as the start and end positions), try to find every other location. For each found, add information to the graph dictionary about which two splits are connected and the distance between them.

In [8]:
def getSplitGraph(start, end, data):
  splits = [start]
  splits.extend(getSplits(data))
  splits.append(end)
  graph = {}
  while splits:
    cur = splits.pop(0)
    for s in splits:
      d = path(cur, s, data)
      if d:
        if graph.get(cur):
          graph[cur][s] = d
        else:
          graph[cur] = {s: d}
        if graph.get(s):
          graph[s][cur] = d
        else:
          graph[s] = {cur: d}
  return graph

### Create a function to find the longest path using the graph data
Go along a path recursively until reaching the end, then compare the length against the current maximum

In [9]:
def longestGraphPath(start, end, graph, path=(), total = 0, m=0):
        path = path + (start,)
        if start == end:
            return max(total, m)
        for node in graph[start]:
            if node not in path:
                m  = longestGraphPath(node, end, graph, path=path, total=total+graph[start][node], m=m)
        return m

### Calculate the longest path

In [10]:
split_graph = getSplitGraph(start, end, data_map)
longestGraphPath(start, end, split_graph)

6526